## Init: configure executors based on our cluster size

docs: 
- https://learn.microsoft.com/en-us/azure/databricks/clusters/cluster-config-best-practices
- https://spark.apache.org/docs/latest/configuration.html

In [0]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import pyspark.sql.functions as F

conf = SparkConf()
conf.setMaster("local").setAppName("My app")
conf.set("spark.executor.instances", "256")
conf.set("spark.executor.cores", "2")
conf.set("spark.dynamicAllocation.initialExecutors",16)


conf.getAll()

source files: 
- raw patient sequences (one record --> one  patient)
- crossjoin files (one record --> one pair of patients)
- computed distances (dlcs)

In [0]:
raw_sampled_100    = '/FileStore/tables/patients-sequences/patients_df_sampled_100.csv'
raw_sampled_1000   = '/FileStore/tables/patients-sequences/patients_df_sampled_1000.csv'
raw_sampled_10000  = '/FileStore/tables/patients-sequences/patients_df_sampled_10000.csv'
# raw_sampled_100000 = '/FileStore/tables/patients-sequences/patients_df_sampled_100000.csv'

crossjoin_partitioned_100    = '/FileStore/tables/patients-sequences/crossjoin_partitioned_100.parquet'
crossjoin_partitioned_1000   = '/FileStore/tables/patients-sequences/crossjoin_partitioned_1000.parquet'
crossjoin_partitioned_10000  = '/FileStore/tables/patients-sequences/crossjoin_partitioned_10000.parquet'
# crossjoin_partitioned_100000 = '/FileStore/tables/patients-sequences/crossjoin_partitioned_100000.parquet'

dlcs_results_100    = '/FileStore/tables/patients-sequences/dlcs_partitioned_100.parquet'
dlcs_results_1000   = '/FileStore/tables/patients-sequences/dlcs_partitioned_1000.parquet'
dlcs_results_10000  = '/FileStore/tables/patients-sequences/dlcs_partitioned_10000.parquet'
# dlcs_results_100000 = '/FileStore/tables/patients-sequences/dlcs_partitioned_100000.parquet'

underpinning dLCS function. in terms of analytics, this is a "heavyweight" UDF that is applied to each record in any of the datasets above

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType, BooleanType, IntegerType
 

In [0]:
## simple example to show UDF registration mechanims

def testF (x,y):
    return len(x) > len(y)

testF_udf = F.udf(testF, BooleanType())

In [0]:
## this is the distance function 
## approx matching of two sequences of tokens
## input tokelist{1,2} are two lists of tokens. 
## return int distance between the two lists

def dlcs(tokenList1, tokenList2):
    # find the length of the lists
    m = len(tokenList1)
    n = len(tokenList2)

    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
 
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif tokenList1[i-1] == tokenList2[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
 
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return m+n- 2 * L[m][n]


dlcs_udf = F.udf(dlcs, IntegerType())

#### set the desired scale for the next experiment

In [0]:
raw_input_dataset = raw_sampled_1000
crossjoin_dataset = crossjoin_partitioned_1000
results_dataset   = dlcs_results_1000

## Spark `DataFrames` approach

so we manage relational tables throughout

the key is to partition the data so that each executor can operate on one partition, concurrently and independently from the others. 
note there are no "reduce" steps here, ideally the set of all distances can be computed in an "embarassingly parallel" fashion

in theory, it should be possible to scale up the resources with the size of the input dataframe. As we will see below, in practice there are limitations

### Data partitioning

firstly, we need to prepare the raw dataset (dataframe). 

This involves two steps:
- computing the crossjoin (cartesian product) of the dataset with itself, producing records of pairs

- partitioning the crossjoin dataset , so that each executor can operate on one partition independently of the others.

We save the partitioned version of the raw dataset to DBFS on the assumption that it will be used multiple times. 

Partitoning is an expensive operation but we only need to perform it once for each input dataset (100, 1000, etc).

#### Steps:
1. create DataFrame from raw csv file
2. lazy compute cartesian product (N^2 records)
3. repartition the product dataset in memory
4. save crossjoin file to dbfs --> **we use the more efficient binary parquet format**

#### 1. create DataFrame from raw csv file

In [0]:
sequences_df = spark.read.csv(raw_input_dataset, header=True, inferSchema=True)
sequences_df.schema

In [0]:
## "clean up" the dataframe schema

sequences_1 = sequences_df.drop('_c0')
sequences_2 = sequences_1.withColumnRenamed('patient_id', 'patient_id_1').withColumnRenamed('read_2', 'read_2_1')

#### 2. lazy compute cartesian product (N^2 records)

crossjoin on the dataframe with itself.

note that this is a "lazy" operation:

In [0]:
sequence_pairs = sequences_1.crossJoin(sequences_2)
sequence_pairs.count() 

#### 3. repartition the product dataset in memory

data partitioning on the product dataset

see https://sparkbyexamples.com/spark/spark-partitioning-understanding

note that repartition is also lazy

several strategies are available to define partitons. A common way is to specify one or more partition columns. This works well if the downstream aggregstions use those columns for "groupby".

In our case it makes sense to use the combination of patient IDs, as we are going to group to group by patient id later, however this will generate N= num of patients partitions, which are too many. 

Thus we limit to 200 random partitions for this exercise.

feel free to experiment with different partitioning strategies, and see what performs better!

In [0]:
partitioned_sequence_pairs = sequence_pairs.repartition(200)

#### 4. save crossjoin file to dbfs

In [0]:
## following command is a Spark _action_: it triggers the actual repartitioning 

## for large datasets, this is an expensive operation, and we only execute it once when preparing the data for repeat analysis

partitioned_sequence_pairs.rdd.getNumPartitions()

In [0]:
partitioned_sequence_pairs.write.mode("overwrite").parquet(crossjoin_dataset)

 ## format("delta").save(sequences_table)

### entry point if partitioned crossjoin datasets are already on FS:

if the crossjoin datasets have been previously saved to DBFS, we can start here by loading them rather than repeat the prep steps above

In [0]:
partitioned_sequence_pairs = spark.read.parquet(crossjoin_dataset)

## Computing distances using UDFs on DataFrames: 

1. compute distances on each partition. this adds a `dlcs` column to each record, holding the distance
- in principle, each computation is independenr from all others
- in Spark this is again lazily evaluated
2. write the resulting dataframe back to disk

In [0]:
partitioned_sequence_pairs_dlcs = partitioned_sequence_pairs.withColumn('dlcs', dlcs_udf('read_2','read_2_1'))

### example aggregations:

no computation has happened yet at this point, but we can illustrate how workers can deliver valuable analytics without the need to centralise the result, i.e., moving all data back to the driver node

Aggregations are optimised by Spark to operate on the partitions, i.e., by having the driver request data from the workers, as needed. 

Thus as long as we never call `collect()` on the computed dataset, we can still be relatively efficient.

#### show K random records from the result dataset

In [0]:
## top-k is relatively inexpensive:

partitioned_sequence_pairs_dlcs.show(10)

#### compute average, min and max distances across the entire dataset

In [0]:
grouped = partitioned_sequence_pairs_dlcs.select('dlcs').groupBy()

In [0]:
grouped.avg().show()

In [0]:
grouped.max().show()

#### avg distance of each patient from all others

In [0]:
## group by first patient, take avg distance

avg_distance_by_patient = partitioned_sequence_pairs_dlcs.groupby('patient_id').agg(F.avg('dlcs').alias('avg_dist')).orderBy('avg_dist', ascending= False)
avg_distance_by_patient.columns

In [0]:
display(avg_distance_by_patient)

In [0]:
display(avg_distance_by_patient.summary())

select patients who have a small average distance from all others

In [0]:
dense_patients = avg_distance_by_patient.filter(avg_distance_by_patient.avg_dist < 65 - 35/2)

dense_patients.count()

In [0]:
display(dense_patients)

### illustrate a SQL interface to querying dataframes

In [0]:
partitioned_sequence_pairs_dlcs.createOrReplaceTempView("patient_distances")

In [0]:
spark.sql("SELECT avg(dlcs) FROM patient_distances").show()

In [0]:
spark.sql("SELECT avg(dlcs) AS avg_dist FROM patient_distances GROUP BY patient_id ORDER BY avg_dist DESC. LIMIT 20").show()

## write result dataset back to disk

writing a partitioned dataset to disk does not require to move its contents back to the driver. 

 -- if the file is naturally partitioned, data may not need to be collected back to the driver

In [0]:
partitioned_sequence_pairs_dlcs.write.parquet(results_dataset, mode='overwrite')

## Appendix: Using rdd.map()

the `rdd.map()` style takes essentially the same time, as under the hood it is actually what is executed on the dataframe

however in this case we need to manually "carry over" the key (patient_id) as the `map()` operation only returns the actual value of the `dlcs()` function

In [0]:
partitioned_sequence_pairs_dlcs  = partitioned_sequence_pairs.rdd.map(lambda row: dlcs(row['read_2'], row['read_2_1'])).collect()

managing RDDs is less intuitive and effectively the Dataframes / SQL API is much more friendly. 

Thus we do not repeat the analytics above here

**WARNING** the `collect()` action forces a complete computation and also transfers all data back to driver. 

This is the most expensive operation, and should be avoided if possible.

In [0]:
partitioned_sequence_pairs_dlcs.collect()

writing the result back to file also forces a complete computation

In [0]:
parquet_file_dlcs = '/FileStore/tables/patients-sequences/patients_df_sampled_1000_dlcs.parquet'
partitioned_sequence_pairs_dlcs.write.parquet(parquet_file_dlcs, mode='overwrite')

## Appendix: using Databricks native Delta Tables

here instead of writing the crossjoin file to parquet (partitioned), we write it to Delta Tables. 

The goal is to check whether this results in faster reads

In [0]:
partitioned_sequences_table = '/FileStore/tables/patients-sequences/patients_df_sampled_10000.delta'

partitioned_sequence_pairs.write.format("delta").mode("overwrite").save(sequences_table)

In [0]:
sequence_pairs_delta = spark.read.format("delta").load(partitioned_sequences_table)

In [0]:
sequence_pairs_delta.count()

In [0]:
partitioned_sequence_pairs_dlcs = partitioned_sequence_pairs.withColumn('dlcs', dlcs_udf('read_2','read_2_1'))

In [0]:
partitioned_sequence_pairs_dlcs.show(10000)

In [0]:
dlcs_delta_file = '/FileStore/tables/patients-sequences/patients_df_sampled_dlcs_partitioned_10000.delta'
partitioned_sequence_pairs_dlcs.write.format("delta").mode("overwrite").save(dlcs_delta_file)